In [154]:
import os

home = os.path.expanduser("~")

db_name=  'colab_main_dataset_21b' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'HR_data')
output_dir = os.path.join(home, 'video_resources', 'sql_database')

In [155]:
from sql_face.alchemy import get_session
session = get_session(output_dir, db_name)

In [156]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sql_face.tables import  QualityImage, FaceImage, CroppedImage, Image, QualityModel, QualityGroup,Detector, EmbeddingModel,EnfsiImage,EnfsiVideoFrame

Ficheros a importar

In [157]:
database = 'XQLFW'

emb_file = os.path.join(input_dir,f'embeddings_{database}_hr.npy')
filenames_file = os.path.join(input_dir,f'filenames_{database}_hr.npy')
quality_file = os.path.join(input_dir,f'results_{database}_hr.csv')

emb = np.load(emb_file) 
filename = np.load(filenames_file)
df_quality = pd.read_csv(quality_file)


In [158]:
df_quality

,Path,Quality
0,/content/CodeFormer/align_hr_XQLFW/154229.png,0.697667
1,/content/CodeFormer/align_hr_XQLFW/151429.png,0.552021
2,/content/CodeFormer/align_hr_XQLFW/153905.png,0.628728
3,/content/CodeFormer/align_hr_XQLFW/157023.png,0.844952
4,/content/CodeFormer/align_hr_XQLFW/162592.png,0.647186
...,...,...
13135,/content/CodeFormer/align_hr_XQLFW/159746.png,0.743560
13136,/content/CodeFormer/align_hr_XQLFW/161929.png,0.586860
13137,/content/CodeFormer/align_hr_XQLFW/158404.png,0.738663
13138,/content/CodeFormer/align_hr_XQLFW/160921.png,0.607221


In [159]:
print(filename.shape,emb.shape, df_quality.shape)


(13140,) (13140, 512) (13140, 2)


In [160]:
df = pd.DataFrame({'Path': filename, 'embedding': emb.tolist()})
df

,Path,embedding
0,/content/CodeFormer/align_hr_XQLFW/154229.png,"[0.2581427991390228, -3.5268006324768066, -1.1..."
1,/content/CodeFormer/align_hr_XQLFW/151429.png,"[-1.0575835704803467, -1.0177922248840332, 1.8..."
2,/content/CodeFormer/align_hr_XQLFW/153905.png,"[-0.058558106422424316, 0.07175542414188385, -..."
3,/content/CodeFormer/align_hr_XQLFW/157023.png,"[0.20844809710979462, -2.281052589416504, -1.7..."
4,/content/CodeFormer/align_hr_XQLFW/162592.png,"[-0.13531062006950378, -0.4333099126815796, -0..."
...,...,...
13135,/content/CodeFormer/align_hr_XQLFW/159746.png,"[-2.8232827186584473, 0.997859001159668, -1.16..."
13136,/content/CodeFormer/align_hr_XQLFW/161929.png,"[1.9345183372497559, -0.753015398979187, -1.89..."
13137,/content/CodeFormer/align_hr_XQLFW/158404.png,"[-0.6367706060409546, -1.958783507347107, -1.1..."
13138,/content/CodeFormer/align_hr_XQLFW/160921.png,"[0.3783136308193207, 0.4765605330467224, 0.194..."


In [161]:
df['CI_id'] = df.Path.apply(lambda x: int(os.path.basename(x).split(".")[0]))
df_quality['CI_id'] = df_quality.Path.apply(lambda x: int(os.path.basename(x).split(".")[0]))

In [162]:
results_data = df.merge(df_quality, on='CI_id')

In [163]:
results_data.shape

(13140, 5)

In [164]:
results_data

,Path_x,embedding,CI_id,Path_y,Quality
0,/content/CodeFormer/align_hr_XQLFW/154229.png,"[0.2581427991390228, -3.5268006324768066, -1.1...",154229,/content/CodeFormer/align_hr_XQLFW/154229.png,0.697667
1,/content/CodeFormer/align_hr_XQLFW/151429.png,"[-1.0575835704803467, -1.0177922248840332, 1.8...",151429,/content/CodeFormer/align_hr_XQLFW/151429.png,0.552021
2,/content/CodeFormer/align_hr_XQLFW/153905.png,"[-0.058558106422424316, 0.07175542414188385, -...",153905,/content/CodeFormer/align_hr_XQLFW/153905.png,0.628728
3,/content/CodeFormer/align_hr_XQLFW/157023.png,"[0.20844809710979462, -2.281052589416504, -1.7...",157023,/content/CodeFormer/align_hr_XQLFW/157023.png,0.844952
4,/content/CodeFormer/align_hr_XQLFW/162592.png,"[-0.13531062006950378, -0.4333099126815796, -0...",162592,/content/CodeFormer/align_hr_XQLFW/162592.png,0.647186
...,...,...,...,...,...
13135,/content/CodeFormer/align_hr_XQLFW/159746.png,"[-2.8232827186584473, 0.997859001159668, -1.16...",159746,/content/CodeFormer/align_hr_XQLFW/159746.png,0.743560
13136,/content/CodeFormer/align_hr_XQLFW/161929.png,"[1.9345183372497559, -0.753015398979187, -1.89...",161929,/content/CodeFormer/align_hr_XQLFW/161929.png,0.586860
13137,/content/CodeFormer/align_hr_XQLFW/158404.png,"[-0.6367706060409546, -1.958783507347107, -1.1...",158404,/content/CodeFormer/align_hr_XQLFW/158404.png,0.738663
13138,/content/CodeFormer/align_hr_XQLFW/160921.png,"[0.3783136308193207, 0.4765605330467224, 0.194...",160921,/content/CodeFormer/align_hr_XQLFW/160921.png,0.607221


In [165]:
results_data.drop(['Path_x','Path_y'] , axis=1, inplace = True)

In [166]:
results_data

,embedding,CI_id,Quality
0,"[0.2581427991390228, -3.5268006324768066, -1.1...",154229,0.697667
1,"[-1.0575835704803467, -1.0177922248840332, 1.8...",151429,0.552021
2,"[-0.058558106422424316, 0.07175542414188385, -...",153905,0.628728
3,"[0.20844809710979462, -2.281052589416504, -1.7...",157023,0.844952
4,"[-0.13531062006950378, -0.4333099126815796, -0...",162592,0.647186
...,...,...,...
13135,"[-2.8232827186584473, 0.997859001159668, -1.16...",159746,0.743560
13136,"[1.9345183372497559, -0.753015398979187, -1.89...",161929,0.586860
13137,"[-0.6367706060409546, -1.958783507347107, -1.1...",158404,0.738663
13138,"[0.3783136308193207, 0.4765605330467224, 0.194...",160921,0.607221


# Datos para la base de datos
Revisar estos datos si cambiamos de database. o hacer las queries para obtenerlos. 

In [167]:
detector_id = 2 # mtcnn-serfiq
embeddingModel_id = 6 #El nuevo
qualityModel_id = 1 #ser_fiq

# Creamos/Actualizamos FaceImage

In [168]:
#buscamos la faceimage y la creamos si no existe
limit = 300
i = limit
for  index, dato in tqdm(results_data.iterrows()):
    #cI = session.query(CroppedImage).join(Image).filter(Image.path == dato.Path,CroppedImage.detector_id==detector_id).one_or_none()
    cI = session.query(CroppedImage).filter(CroppedImage.croppedImage_id == dato.CI_id,CroppedImage.detector_id==detector_id).one_or_none()

    if cI:
        FI = session.query(FaceImage).filter(FaceImage.croppedImages == cI,FaceImage.embeddingModel_id == embeddingModel_id).one_or_none()
        if FI:
            FI.embeddings=dato.embedding

        else:
            new_face_image = FaceImage(croppedImage_id=cI.croppedImage_id,
            embeddingModel_id=embeddingModel_id,
            embeddings=dato.embedding,
            confusion_score=None)
            session.add(new_face_image)
        i-=1
        #session.commit()        

    else:
        print (f'{dato.Path} no encontrado')

    if not i:
        i=limit
        session.commit()

session.commit()
session.close()



13140it [00:30, 424.18it/s]


# Creamos / actualizamos QualityImage

In [169]:
#buscamos la qualityimage y la creamos si no existe
limit = 100
i = limit
for  index, dato in tqdm(results_data.iterrows()):
    FI = session.query(FaceImage).filter(FaceImage.embeddingModel_id == embeddingModel_id) \
                .join(CroppedImage).filter(CroppedImage.croppedImage_id == dato.CI_id).one_or_none()
#                .join(Image).filter(Image.path == dato.Path)

    if FI:
        QI = session.query(QualityImage).filter(QualityImage.faceImages == FI, QualityImage.qualityModel_id == qualityModel_id).one_or_none()
        if QI:
            QI.quality=dato.Quality

        else:
            new_quality_image = QualityImage(faceImage_id = FI.faceImage_id,
            qualityModel_id = qualityModel_id,
            quality=dato.Quality)
            session.add(new_quality_image)
        i-=1
        #session.commit()        

    else:
        print (dato.Path + 'no encontrado')

    if not i:
        i=limit
        session.commit()

session.commit()
session.close()





13140it [00:24, 530.68it/s]
